In [4]:
import caffe
import numpy as np

In [5]:
model_filename = './Data/yolo_train_val.prototxt'
yoloweight_filename = './Data/yolov3.weights'
caffemodel_filename = './Data/yolo_train_val.caffemodel'

In [6]:
def gencaffemodel(model_filename, yoloweight_filename, caffemodel_filename):
    print('model file is ' + model_filename)
    print('weight file is ' + yoloweight_filename)
    print('caffemodel file is ' + caffemodel_filename)
    
    net = caffe.Net(model_filename, caffe.TEST)
    params = net.params.keys()
    
    # read weights from file and assign to the network
    netWeightsInt = np.fromfile(yoloweight_filename, dtype=np.int32)
    transFlag = (netWeightsInt[0]>1000 or netWeightsInt[1]>1000) # transpose flag, the first 4 entries are major, minor, revision and net.seen
    print(transFlag) 
    
    netWeightsFloat = np.fromfile(yoloweight_filename, dtype=np.float32)
    netWeights = netWeightsFloat[4:] # start from the 5th entry, the first 4 entries are major, minor, revision and net.seen
    print(netWeights.shape)
    count = 0
    for pr in params:
        lidx = list(net._layer_names).index(pr)
        layer = net.layers[lidx]
        if count == netWeights.shape[0] and (layer.type != 'BatchNorm' and layer.type != 'Scale'):
            print("WARNING: no weights left for %s" % pr)
            break
        if layer.type == 'Convolution':
            print(pr+"(conv)")
            # bias
            if len(net.params[pr]) > 1:
                bias_dim = net.params[pr][1].data.shape
            else:
                bias_dim = (net.params[pr][0].data.shape[0], )
            biasSize = np.prod(bias_dim)
            conv_bias = np.reshape(netWeights[count:count+biasSize], bias_dim)
            if len(net.params[pr]) > 1:
                assert(bias_dim == net.params[pr][1].data.shape)
                net.params[pr][1].data[...] = conv_bias
                conv_bias = None
            count = count + biasSize
            # batch_norm
            next_layer = net.layers[lidx+1]
            if next_layer.type == 'BatchNorm':
                bn_dims = (3, net.params[pr][0].data.shape[0])
                bnSize = np.prod(bn_dims)
                batch_norm = np.reshape(netWeights[count:count+bnSize], bn_dims)
                count = count + bnSize
            # weights
            dims = net.params[pr][0].data.shape
            weightSize = np.prod(dims)
            net.params[pr][0].data[...] = np.reshape(netWeights[count:count+weightSize], dims)
            count = count + weightSize
        elif layer.type == 'InnerProduct':
            print(pr+"(fc)")
            # bias
            biasSize = np.prod(net.params[pr][1].data.shape)
            net.params[pr][1].data[...] = np.reshape(netWeights[count:count+biasSize], net.params[pr][1].data.shape)
            count = count + biasSize
            # weights
            dims = net.params[pr][0].data.shape
            weightSize = np.prod(dims)
            if transFlag:
                net.params[pr][0].data[...] = np.reshape(netWeights[count:count+weightSize], (dims[1], dims[0])).transpose()
            else:
                net.params[pr][0].data[...] = np.reshape(netWeights[count:count+weightSize], dims)
            count = count + weightSize
        elif layer.type == 'BatchNorm':
            print(pr+"(batchnorm)")
            net.params[pr][0].data[...] = batch_norm[1]	# mean
            net.params[pr][1].data[...] = batch_norm[2]	# variance
            net.params[pr][2].data[...] = 1.0	# scale factor
        elif layer.type == 'Scale':
            print(pr+"(scale)")
            net.params[pr][0].data[...] = batch_norm[0]	# scale
            batch_norm = None
            if len(net.params[pr]) > 1:
                net.params[pr][1].data[...] = conv_bias	# bias
                conv_bias = None
        else:
            print("WARNING: unsupported layer, "+pr)
    if np.prod(netWeights.shape) != count:
        print("ERROR: size mismatch: %d" % count)
    net.save(caffemodel_filename)

In [7]:
gencaffemodel(model_filename, yoloweight_filename, caffemodel_filename)

model file is ./Data/yolo_train_val.prototxt
weight file is ./Data/yolov3.weights
caffemodel file is ./Data/yolo_train_val.caffemodel


I0127 13:35:26.399883  7058 upgrade_proto.cpp:69] Attempting to upgrade input file specified using deprecated input fields: ./Data/yolo_train_val.prototxt
I0127 13:35:26.399932  7058 upgrade_proto.cpp:72] Successfully upgraded file specified using deprecated input fields.
W0127 13:35:26.399940  7058 upgrade_proto.cpp:74] Note that future Caffe releases will only support input layers and not input fields.
I0127 13:35:26.400074  7058 net.cpp:53] Initializing net from parameters: 
name: "YOLONet"
state {
  phase: TEST
  level: 0
}
layer {
  name: "input"
  type: "Input"
  top: "data"
  input_param {
    shape {
      dim: 1
      dim: 3
      dim: 448
      dim: 448
    }
  }
}
layer {
  name: "conv1"
  type: "Convolution"
  bottom: "data"
  top: "conv1"
  convolution_param {
    num_output: 64
    pad: 3
    kernel_size: 7
    stride: 2
    weight_filler {
      type: "gaussian"
      std: 0.01
    }
    bias_filler {
      type: "constant"
      value: 0
    }
  }
}
layer {
  name: "rel

False
(62001758,)
conv1(conv)
conv2(conv)
conv3(conv)
conv4(conv)
conv5(conv)
conv6(conv)
conv7(conv)
conv8(conv)
conv9(conv)
conv10(conv)
conv11(conv)
conv12(conv)
conv13(conv)
conv14(conv)
conv15(conv)
conv16(conv)
conv17(conv)
conv18(conv)
conv19(conv)
conv20(conv)
conv21(conv)
conv22(conv)
conv23(conv)
conv24(conv)
fc25(fc)


ValueError: cannot reshape array of size 1841694 into shape (4096,50176)

In [8]:
model_filename = './Data/yolo_small_train_val.prototxt'
yoloweight_filename = './Data/yolov3.weights'
caffemodel_filename = './Data/yolo_small_train_val.caffemodel'
gencaffemodel(model_filename, yoloweight_filename, caffemodel_filename)

model file is ./Data/yolo_small_train_val.prototxt
weight file is ./Data/yolov3.weights
caffemodel file is ./Data/yolo_small_train_val.caffemodel


I0127 13:36:00.274158  7058 upgrade_proto.cpp:69] Attempting to upgrade input file specified using deprecated input fields: ./Data/yolo_small_train_val.prototxt
I0127 13:36:00.274188  7058 upgrade_proto.cpp:72] Successfully upgraded file specified using deprecated input fields.
W0127 13:36:00.274194  7058 upgrade_proto.cpp:74] Note that future Caffe releases will only support input layers and not input fields.
I0127 13:36:00.274341  7058 net.cpp:53] Initializing net from parameters: 
name: "YOLONet"
state {
  phase: TEST
  level: 0
}
layer {
  name: "input"
  type: "Input"
  top: "data"
  input_param {
    shape {
      dim: 1
      dim: 3
      dim: 448
      dim: 448
    }
  }
}
layer {
  name: "conv1"
  type: "Convolution"
  bottom: "data"
  top: "conv1"
  convolution_param {
    num_output: 64
    pad: 3
    kernel_size: 7
    stride: 2
    weight_filler {
      type: "gaussian"
      std: 0.01
    }
    bias_filler {
      type: "constant"
      value: 0
    }
  }
}
layer {
  name

False
(62001758,)
conv1(conv)
conv2(conv)
conv3(conv)
conv4(conv)
conv5(conv)
conv6(conv)
conv7(conv)
conv8(conv)
conv9(conv)
conv10(conv)
conv11(conv)
conv12(conv)
conv13(conv)
conv14(conv)
conv15(conv)
conv16(conv)
conv17(conv)
conv18(conv)
conv19(conv)
conv20(conv)
conv21(conv)
conv22(conv)
conv23(conv)
conv24(conv)
fc25(fc)


ValueError: cannot reshape array of size 1845278 into shape (512,50176)

In [ ]:
model_filename = './Data/yolo_tiny_train_val.prototxt'
yoloweight_filename = './Data/yolov3.weights'
caffemodel_filename = './Data/yolo_tiny_train_val.caffemodel'
gencaffemodel(model_filename, yoloweight_filename, caffemodel_filename)

model file is ./Data/yolo_tiny_train_val.prototxt
weight file is ./Data/yolov3.weights
caffemodel file is ./Data/yolo_tiny_train_val.caffemodel


I0127 13:36:15.589802  7058 upgrade_proto.cpp:69] Attempting to upgrade input file specified using deprecated input fields: ./Data/yolo_tiny_train_val.prototxt
I0127 13:36:15.592324  7058 upgrade_proto.cpp:72] Successfully upgraded file specified using deprecated input fields.
W0127 13:36:15.592335  7058 upgrade_proto.cpp:74] Note that future Caffe releases will only support input layers and not input fields.
I0127 13:36:15.592344  7058 upgrade_proto.cpp:79] Attempting to upgrade batch norm layers using deprecated params: ./Data/yolo_tiny_train_val.prototxt
I0127 13:36:15.592357  7058 upgrade_proto.cpp:82] Successfully upgraded batch norm layers using deprecated params.
I0127 13:36:15.593261  7058 net.cpp:296] The NetState phase (1) differed from the phase (0) specified by a rule in layer bn1
I0127 13:36:15.593297  7058 net.cpp:296] The NetState phase (1) differed from the phase (0) specified by a rule in layer bn2
I0127 13:36:15.593314  7058 net.cpp:296] The NetState phase (1) differe